## Setting Up:

In [ ]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), '../../')) # Add root of repo to import MBM
import csv

import pandas as pd
import warnings
from tqdm.notebook import tqdm
import re
import matplotlib.pyplot as plt
import seaborn as sns
from cmcrameri import cm
import xarray as xr
import massbalancemachine as mbm
from collections import defaultdict
import logging
import torch.nn as nn
from skorch.helper import SliceDataset
from datetime import datetime
from skorch.callbacks import EarlyStopping, LRScheduler, Checkpoint
import itertools
import random

import pickle 
from collections import Counter
import ast
from torch.utils.data import Subset
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset


from scripts.helpers import *
from scripts.glamos_preprocess import *
from scripts.plots import *
from scripts.config_CH import *
from scripts.nn_helpers import *
from scripts.xgb_helpers import *
from scripts.geodata import *
from scripts.NN_networks import *
from scripts.geodata_plots import *
from scripts.lstm import *

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

cfg = mbm.SwitzerlandConfig()

In [ ]:
# Plot styles:
path_style_sheet = 'scripts/example.mplstyle'
plt.style.use(path_style_sheet)
colors = get_cmap_hex(cm.batlow, 10)
color_dark_blue = colors[0]
color_pink = '#c51b7d'

# RGI Ids:
# Read rgi ids:
rgi_df = pd.read_csv(cfg.dataPath + path_glacier_ids, sep=',')
rgi_df.rename(columns=lambda x: x.strip(), inplace=True)
rgi_df.sort_values(by='short_name', inplace=True)
rgi_df.set_index('short_name', inplace=True)

vois_climate = [
    't2m',
    'tp',
    'slhf',
    'sshf',
    'ssrd',
    'fal',
    'str',
]

vois_topographical = [
    "aspect_sgi",
    "slope_sgi",
    "hugonnet_dhdt",
    "consensus_ice_thickness",
    "millan_v",
]

In [ ]:
seed_all(cfg.seed)

print("Using seed:", cfg.seed)

if torch.cuda.is_available():
    print("CUDA is available")
    free_up_cuda()
else:
    print("CUDA is NOT available")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Read GL data:

In [ ]:
data_glamos = getStakesData(cfg)

# Capitalize glacier names:
glacierCap = {}
for gl in data_glamos['GLACIER'].unique():
    if isinstance(gl, str):  # Ensure the glacier name is a string
        if gl.lower() == 'claridenu':
            glacierCap[gl] = 'Clariden_U'
        elif gl.lower() == 'claridenl':
            glacierCap[gl] = 'Clariden_L'
        else:
            glacierCap[gl] = gl.capitalize()
    else:
        print(f"Warning: Non-string glacier name encountered: {gl}")

# drop taelliboden and plainemorte if in there
if 'taelliboden' in data_glamos['GLACIER'].unique():
    data_glamos = data_glamos[data_glamos['GLACIER'] != 'taelliboden']
if 'plainemorte' in data_glamos['GLACIER'].unique():
    data_glamos = data_glamos[data_glamos['GLACIER'] != 'plainemorte']

print('-------------------')
print('Number of glaciers:', len(data_glamos['GLACIER'].unique()))
print('Number of winter and annual samples:', len(data_glamos))
print('Number of annual samples:',
      len(data_glamos[data_glamos.PERIOD == 'annual']))
print('Number of winter samples:',
      len(data_glamos[data_glamos.PERIOD == 'winter']))


## Input data:
### Input dataset:

In [ ]:
# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Transform data to monthly format (run or load data):
paths = {
    'csv_path': cfg.dataPath + path_PMB_GLAMOS_csv,
    'era5_climate_data':
    cfg.dataPath + path_ERA5_raw + 'era5_monthly_averaged_data.nc',
    'geopotential_data':
    cfg.dataPath + path_ERA5_raw + 'era5_geopotential_pressure.nc',
    'radiation_save_path': cfg.dataPath + path_pcsr + 'zarr/'
}
RUN = False
dataloader_gl = process_or_load_data(
    run_flag=RUN,
    data_glamos=data_glamos,
    paths=paths,
    cfg=cfg,
    vois_climate=vois_climate,
    vois_topographical=vois_topographical,
    output_file='CH_wgms_dataset_monthly_NN.csv')
data_monthly = dataloader_gl.data

data_monthly['GLWD_ID'] = data_monthly.apply(
    lambda x: mbm.data_processing.utils.get_hash(f"{x.GLACIER}_{x.YEAR}"),
    axis=1)
data_monthly['GLWD_ID'] = data_monthly['GLWD_ID'].astype(str)

dataloader_gl = mbm.dataloader.DataLoader(cfg,
                                          data=data_monthly,
                                          random_seed=cfg.seed,
                                          meta_data_columns=cfg.metaData)

data_annual = dataloader_gl.data[dataloader_gl.data.PERIOD == 'annual']

# print mean and std of N_MONTHS
print('Mean number of months:', data_annual.N_MONTHS.mean())
print('Std number of months:', data_annual.N_MONTHS.std())

# same for winter
data_winter = dataloader_gl.data[dataloader_gl.data.PERIOD == 'winter']
print('Mean number of months (winter):', data_winter.N_MONTHS.mean())
print('Std number of months (winter):', data_winter.N_MONTHS.std())

## Blocking on glaciers:

In [ ]:
# Ensure all test glaciers exist in the dataset
existing_glaciers = set(dataloader_gl.data.GLACIER.unique())
missing_glaciers = [g for g in TEST_GLACIERS if g not in existing_glaciers]

if missing_glaciers:
    print(
        f"Warning: The following test glaciers are not in the dataset: {missing_glaciers}"
    )

# Define training glaciers correctly
train_glaciers = [i for i in existing_glaciers if i not in TEST_GLACIERS]

data_test = dataloader_gl.data[dataloader_gl.data.GLACIER.isin(TEST_GLACIERS)]
print('Size of monthly test data:', len(data_test))

data_train = dataloader_gl.data[dataloader_gl.data.GLACIER.isin(
    train_glaciers)]
print('Size of monthly train data:', len(data_train))

if len(data_train) == 0:
    print("Warning: No training data available!")
else:
    test_perc = (len(data_test) / len(data_train)) * 100
    print('Percentage of test size: {:.2f}%'.format(test_perc))

# Number of annual versus winter measurements:
print('-------------\nTrain:')
print('Number of monthly winter and annual samples:', len(data_train))
print('Number of monthly annual samples:',
      len(data_train[data_train.PERIOD == 'annual']))
print('Number of monthly winter samples:',
      len(data_train[data_train.PERIOD == 'winter']))

# Same for test
data_test_annual = data_test[data_test.PERIOD == 'annual']
data_test_winter = data_test[data_test.PERIOD == 'winter']

print('Test:')
print('Number of monthly winter and annual samples:', len(data_test))
print('Number of monthly annual samples:', len(data_test_annual))
print('Number of monthly winter samples:', len(data_test_winter))

print('Total:')
print('Number of monthly rows:', len(dataloader_gl.data))
print('Number of annual rows:',
      len(dataloader_gl.data[dataloader_gl.data.PERIOD == 'annual']))
print('Number of winter rows:',
      len(dataloader_gl.data[dataloader_gl.data.PERIOD == 'winter']))

# same for original data:
print('-------------\nIn annual format:')
print('Number of annual train rows:',
      len(data_glamos[data_glamos.GLACIER.isin(train_glaciers)]))
print('Number of annual test rows:',
      len(data_glamos[data_glamos.GLACIER.isin(TEST_GLACIERS)]))

splits, test_set, train_set = get_CV_splits(dataloader_gl,
                                            test_split_on='GLACIER',
                                            test_splits=TEST_GLACIERS,
                                            random_state=cfg.seed)

print('Test glaciers: ({}) {}'.format(len(test_set['splits_vals']),
                                      test_set['splits_vals']))
test_perc = (len(test_set['df_X']) / len(train_set['df_X'])) * 100
print('Percentage of test size: {:.2f}%'.format(test_perc))
print('Size of test set:', len(test_set['df_X']))
print('Train glaciers: ({}) {}'.format(len(train_set['splits_vals']),
                                       train_set['splits_vals']))
print('Size of train set:', len(train_set['df_X']))

# Validation and train split:
data_train = train_set['df_X']
data_train['y'] = train_set['y']

data_test = test_set['df_X']
data_test['y'] = test_set['y']

## LSTM:

In [ ]:
MONTHLY_COLS = ['t2m', 'tp', 'slhf', 'sshf', 'ssrd', 'fal', 'str', 'pcsr']
STATIC_COLS = [
    'ELEVATION_DIFFERENCE', 'aspect_sgi', 'slope_sgi', 'hugonnet_dhdt',
    'consensus_ice_thickness', 'millan_v'
]

HYDRO_MONTHS = [
    'oct', 'nov', 'dec', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul',
    'aug', 'sep'
]
HYDRO_POS = {m: i for i, m in enumerate(HYDRO_MONTHS)}

feature_columns = MONTHLY_COLS + STATIC_COLS

### Build LSTM dataloaders:

In [ ]:
df_train = data_train.copy()
df_train['PERIOD'] = df_train['PERIOD'].str.strip().str.lower()

df_test = data_test.copy()
df_test['PERIOD'] = df_test['PERIOD'].str.strip().str.lower()


In [ ]:
# --- build train dataset from dataframe ---
ds_train = mbm.data_processing.MBSequenceDataset.from_dataframe(
    df_train, MONTHLY_COLS, STATIC_COLS, HYDRO_POS, expect_target=True)

# --- make train/val loaders and get split indices (scalers fit on TRAIN, applied to whole ds) ---
train_dl, val_dl, train_idx, val_idx = ds_train.make_loaders(
    val_ratio=0.2,
    batch_size_train=64,
    batch_size_val=128,
    seed=42,
    fit_and_transform=
    True,  # fit scalers on TRAIN and transform Xm/Xs/y in-place
    shuffle_train=True,
)

# sanity checks
print("y mean/std (train):", float(ds_train.y_mean), float(ds_train.y_std))
for name, arr in [("Xm", ds_train.Xm), ("Xs", ds_train.Xs), ("y", ds_train.y)]:
    print(name, "has NaN?", bool(torch.isnan(arr).any().item()))
print("Train counts:", int(ds_train.iw[train_idx].sum()), "winter |",
      int(ds_train.ia[train_idx].sum()), "annual")
print("Val   counts:", int(ds_train.iw[val_idx].sum()), "winter |",
      int(ds_train.ia[val_idx].sum()), "annual")

# --- build test dataset and loader (reuses train scalers) ---
ds_test = mbm.data_processing.MBSequenceDataset.from_dataframe(
    df_test,
    MONTHLY_COLS,
    STATIC_COLS,
    HYDRO_POS,
    expect_target=True  # you said test has real targets
)
test_dl = mbm.data_processing.MBSequenceDataset.make_test_loader(
    ds_test, ds_train, batch_size=128)

### Split train & val:

### Define & train model:

In [ ]:
Fm = ds_train.Xm.shape[-1]
Fs = ds_train.Xs.shape[-1]
model = mbm.models.LSTM_MB(Fm,
                           Fs,
                           hidden_size=128,
                           num_layers=1,
                           bidirectional=True,
                           dropout=0.1,
                           static_hidden=64,
                           static_layers=2).to(device)

In [ ]:
TRAIN = False
if TRAIN:
    history, best_val, best_state = model.train_loop(
        device=device,
        train_dl=train_dl,
        val_dl=val_dl,
        epochs=150,
        lr=1e-3,
        weight_decay=1e-4,
        clip_val=1.0,
        # scheduler
        sched_factor=0.5,
        sched_patience=6,
        sched_threshold=0.01,
        sched_threshold_mode="rel",
        sched_cooldown=1,
        sched_min_lr=1e-6,
        # early stopping
        es_patience=20,
        es_min_delta=1e-4,
        # logging
        log_every=5,
        verbose=True,
        # optional checkpoint
        save_best_path="models/best_lstm_mb.pt",
    )
else:
    # Load the best weights
    best_state = torch.load("models/best_lstm_mb.pt", map_location=device)
    model.load_state_dict(best_state)

### Make predictions:

#### Evaluate on test:

In [ ]:
test_metrics, test_df_preds = mbm.models.LSTM_MB.evaluate_with_preds(
    model, device, test_dl, ds_test)
print("TEST metrics:", test_metrics)
print(test_df_preds.head())

In [ ]:
scores_annual, scores_winter = compute_seasonal_scores(test_df_preds,
                                                       target_col='target',
                                                       pred_col='pred')

print("Annual scores:", scores_annual)
print("Winter scores:", scores_winter)

fig = plot_predictions_summary(grouped_ids=test_df_preds,
                               scores_annual=scores_annual,
                               scores_winter=scores_winter,
                               predVSTruth=predVSTruth,
                               plotMeanPred=plotMeanPred,
                               ax_xlim=(-8, 6),
                               ax_ylim=(-8, 6))

In [ ]:
gl_per_el = data_glamos[data_glamos.PERIOD == 'annual'].groupby(
    ['GLACIER'])['POINT_ELEVATION'].mean()
gl_per_el = gl_per_el.sort_values(ascending=False)

test_gl_per_el = gl_per_el[TEST_GLACIERS].sort_values().index

fig, axs = plt.subplots(3, 3, figsize=(20, 15), sharex=True)

PlotIndividualGlacierPredVsTruth(test_df_preds,
                                 axs=axs,
                                 color_annual=color_dark_blue,
                                 color_winter=color_pink,
                                 custom_order=test_gl_per_el)


## Grid search:

In [ ]:
param_space = {
    "hidden_size": [64, 128, 256],
    "num_layers": [1, 2],
    "bidirectional": [True, False],
    "dropout": [0.0, 0.1, 0.3],
    "static_hidden": [32, 64, 128],
    "lr": [1e-3, 5e-4, 1e-4],
    "weight_decay": [1e-4, 1e-5],
    "batch_size": [32, 64, 128],
}


def sample_param_combinations(space, n_samples):
    all_keys = list(space.keys())
    all_vals = list(space.values())
    all_combinations = list(itertools.product(*all_vals))
    random.shuffle(all_combinations)
    sampled = all_combinations[:n_samples]
    return [dict(zip(all_keys, vals)) for vals in sampled]


sampled_params = sample_param_combinations(param_space, n_samples=100)
sampled_params


In [ ]:
# make sure logs dir exists
os.makedirs("logs", exist_ok=True)
log_filename = f'logs/lstm_param_search_progress_{datetime.now().strftime("%Y-%m-%d")}_SEB_all_OGGM.csv'

# create log with header
with open(log_filename, mode='w', newline='') as log_file:
    writer = csv.DictWriter(log_file,
                            fieldnames=list(sampled_params[0].keys()) +
                            ['valid_loss', 'test_rmse_a', 'test_rmse_w'])
    writer.writeheader()

results = []
best_overall = {"val": float('inf'), "row": None, "params": None}

for i, params in enumerate(sampled_params):
    print(f"\n--- Running config {i+1}/{len(sampled_params)} ---")
    print(params)

    # Build model
    model = mbm.models.LSTM_MB(
        Fm=ds_train.Xm.shape[-1],
        Fs=ds_train.Xs.shape[-1],
        hidden_size=params['hidden_size'],
        num_layers=params['num_layers'],
        bidirectional=params['bidirectional'],
        dropout=params['dropout'],
        static_hidden=params['static_hidden'],
    ).to(device)

    history, best_val, best_state = model.train_loop(
        device=device,
        train_dl=train_dl,
        val_dl=val_dl,
        epochs=150,
        lr=params['lr'],
        weight_decay=params['weight_decay'],
        clip_val=1.0,
        # scheduler
        sched_factor=0.5,
        sched_patience=6,
        sched_threshold=0.01,
        sched_threshold_mode="rel",
        sched_cooldown=1,
        sched_min_lr=1e-6,
        # early stopping
        es_patience=15,
        es_min_delta=1e-4,
        # logging
        log_every=5,
        verbose=False,
        # optional checkpoint
        save_best_path="models/best_lstm_mb_gs.pt",
    )

    # Load the best weights
    best_state = torch.load("models/best_lstm_mb_gs.pt", map_location=device)
    model.load_state_dict(best_state)

    # Evaluate on test for this config
    test_metrics, test_df_preds = mbm.models.LSTM_MB.evaluate_with_preds(
        model, device, test_dl, ds_test)
    test_rmse_a, test_rmse_w = test_metrics['RMSE_annual'], test_metrics[
        'RMSE_winter']

    # Log a row for this config
    row = {
        **params, 'valid_loss': float(best_val),
        'test_rmse_a': float(test_rmse_a),
        'test_rmse_w': float(test_rmse_w)
    }
    with open(log_filename, mode='a', newline='') as log_file:
        writer = csv.DictWriter(log_file, fieldnames=list(row.keys()))
        writer.writerow(row)

    results.append(row)

    # Track overall best by validation loss (or swap to test RMSE if you prefer)
    if best_val < best_overall['val']:
        best_overall = {"val": best_val, "row": row, "params": params}

print("\n=== Best config by validation loss ===")
print(best_overall['params'])
print(best_overall['row'])
